This tutorial will show you how to perform ensemble pocket matching between two proteins.

In [1]:
import mdtraj
import numpy as np
import alphaspace2 as al

For convenience, we pre aligned the Bcl-2 (PDB ID: 2XA0) and Bcl-xL (PDB ID: 3PL7) structures and prepared .pdbqt files for the protein part.

In [4]:
bcl2_prot, bcl2_lig = mdtraj.load('Example_proteins/Bcl2_BAX/prot.pdb'),mdtraj.load('Example_proteins/Bcl2_BAX/lig.pdb')
bclxl_prot, bclxl_lig = mdtraj.load('Example_proteins/BclXL_BAX/prot.pdb'),mdtraj.load('Example_proteins/BclXL_BAX/lig.pdb')
al.annotateVinaAtomTypes(pdbqt="Example_proteins/Bcl2_BAX/prot.pdbqt", receptor=bcl2_prot)
al.annotateVinaAtomTypes(pdbqt="Example_proteins/BclXL_BAX/prot.pdbqt", receptor=bclxl_prot)

In [5]:
ss_bcl2 = al.Snapshot()
ss_bcl2.run(bcl2_prot, bcl2_lig)
ss_bclxl = al.Snapshot()
ss_bclxl.run(bclxl_prot, bclxl_lig)


In [11]:
# Combine two snapshots into a trajectory object
bcl2_bclxl = al.Trajectory(snapshots=[ss_bcl2,ss_bclxl])
# call generate dpocket function to cluster the pockets into d pocket
bcl2_bclxl.gen_dpockets(clust_distance=4.7)

centroid_pockets_to_write=[[],[]]
betas_pockets_to_write = [[],[]]

contact_pockets = []

#Sort dpockets by scores
dps = sorted([dp for dp in bcl2_bclxl.dpockets],key=lambda i:sum(i.scores))

print('dpindex',' ','dp 1 score',"\t",'dp 2 score')
for dp_idx,dp in enumerate(dps):
    pockets = list(dp.pockets)
    if pockets[0].isContact or pockets[1].isContact:
        contact_pockets.append(dp_idx)
        print(dp_idx,"\t",pockets[0].score,"\t",pockets[1].score)
        if pockets[0].score!=0:
            centroid_pockets_to_write[0].append((dp_idx,pockets[0].centroid))
            betas_pockets_to_write[0].append((dp_idx,[b.xyz for b in pockets[0].betas]))
        if pockets[1].score!=0:
            centroid_pockets_to_write[1].append((dp_idx,pockets[1].centroid))
            betas_pockets_to_write[1].append((dp_idx,[b.xyz for b in pockets[1].betas]))




dpindex   dp 1 score 	 dp 2 score
0 	 -5.36221 	 -5.85937
2 	 -2.92216 	 -5.56242
3 	 -5.44176 	 -2.35957
8 	 -6.21846 	 0
10 	 -4.61358 	 -1.33412
11 	 -4.69066 	 -1.16455
29 	 -2.22722 	 -1.47425
42 	 -1.47566 	 -1.42387
43 	 -1.09942 	 -1.78696
54 	 -1.28934 	 -0.976428
56 	 -1.22297 	 -0.943039
59 	 0 	 -2.10167
60 	 0 	 -2.06812
85 	 -0.629927 	 -0.737148
89 	 0 	 -1.22415
96 	 -0.375574 	 -0.638932
106 	 -0.69141 	 0
123 	 0 	 -0.394766
125 	 -0.360147 	 0
